In [ ]:
import gc
import os
import sys
import pylab
import zipfile
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# TensorFlow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator, load_img

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# Google Colab
# path_to_file = '/content/gdrive/My Drive/mouth-shape-classification'
# Local
path_to_file = os.path.abspath(os.curdir)
path_to_file

In [ ]:
# For Google Colaboratory
if 'google.colab' in sys.modules:
    # Mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    path_to_file = path_to_file
    print(path_to_file)
    # Move to Google Drive directory
    os.chdir(path_to_file)

## Dataset

### Load dataset

In [ ]:
split = 0.2
seed = 19260817
batch_size = 16
img_size = (224, 224)

train_ds = keras.utils.image_dataset_from_directory(
    path_to_file + '/dataset/train',
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=batch_size,
    image_size=img_size,
    shuffle=True,
    seed=seed,
    validation_split=split,
    subset='training',
)
val_ds = keras.utils.image_dataset_from_directory(
    path_to_file + '/dataset/train',
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=batch_size,
    image_size=img_size,
    shuffle=True,
    seed=seed,
    validation_split=split,
    subset='validation',
)

### Visualize dataset

In [ ]:
label_names = {0: 'big', 1: 'normal', 2: 'small'}

fig, ax = plt.subplots(figsize=(15, 7))
for img, label in train_ds.take(1):
    for i in range(12):
        plt.subplot(3, 4, i + 1)
        plt.imshow(img.numpy()[i] / 255)
        plt.axis("off")
        plt.title("label: " + label_names[label.numpy()[i]])
plt.show()

## VGG16 Model

### Plot history

In [ ]:
def plot_history(history, title=''):  
    if title != '':
        title += ' '
    # Ploting the Loss and Accuracy Curves
    fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (16, 6))
    # Loss
    sns.lineplot(data = history.history['loss'], label = 'Training Loss', ax = ax[0])
    sns.lineplot(data = history.history['val_loss'], label = 'Validation Loss', ax = ax[0])
    ax[0].legend(loc = 'upper right')
    ax[0].set_title(title + 'Loss')
    # Accuracy
    sns.lineplot(data = history.history['accuracy'], label = 'Training Accuracy', ax = ax[1])
    sns.lineplot(data = history.history['val_accuracy'], label = 'Validation Accuracy', ax = ax[1])
    ax[1].legend(loc = 'lower right')
    ax[1].set_title(title + 'Accuracy')

### Augmentation

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomTranslation(0.14, 0.14),
        layers.RandomZoom(0.2),
        layers.RandomContrast(0.2),
    ]
)

### Save best callback

In [ ]:
def save_best(name:str='vgg16_fine_tuning', patient=3):
    CB = [
        keras.callbacks.ModelCheckpoint(
        filepath=name + ".keras",
        save_best_only=True,
        monitor="val_loss"),

        keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.0005,
        patience=patient)
    ]
    return CB

### Load VGG16 model

In [ ]:
vgg16_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)

# vgg16_base.trainable = True
vgg16_base.trainable = False

print("This is the number of trainable weights before freezing the conv base:", len(vgg16_base.trainable_weights))

vgg16_base.summary()

In [ ]:
inputs = keras.Input(shape=img_size+(3,))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = vgg16_base(x)
x = layers.Flatten()(x)
x = layers.Dense(512)(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512)(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512)(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512)(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512)(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512)(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512)(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(3, activation="softmax")(x)
vgg16_model = keras.Model(inputs, outputs)
vgg16_model.compile(optimizer="adam",
                    loss="sparse_categorical_crossentropy",
                    metrics=["accuracy"])
vgg16_model.summary()

In [ ]:
history_vgg16 = vgg16_model.fit(
    train_ds,
    epochs=200,
    validation_data=val_ds,
    callbacks=save_best('vgg16_fine_tuning', 3)
    )
vgg16_model = keras.models.load_model("vgg16_fine_tuning.keras")

In [ ]:
plot_history(history_vgg16, 'vgg16')

### Load VGG19 model

In [ ]:
vgg19_base = keras.applications.vgg19.VGG19(
    weights="imagenet",
    include_top=False)

# vgg19_base.trainable = True
vgg19_base.trainable = False

print("This is the number of trainable weights before freezing the conv base: ", len(vgg19_base.trainable_weights))

vgg19_base.summary()

In [ ]:
inputs = keras.Input(shape=img_size+(3,))
x = data_augmentation(inputs)
x = keras.applications.vgg19.preprocess_input(x)
x = vgg19_base(x)
x = layers.Flatten()(x)
x = layers.Dense(512)(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512)(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512)(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512)(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512)(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512)(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512)(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(3, activation="softmax")(x)
vgg19_model = keras.Model(inputs, outputs)
vgg19_model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
vgg19_model.summary()

In [ ]:
history_vgg19 = vgg19_model.fit(
    train_ds,
    epochs=100,
    validation_data=val_ds,
    callbacks=save_best('vgg19_fine_tuning', 3)
    )
vgg19_model = keras.models.load_model("vgg19_fine_tuning.keras")

In [ ]:
plot_history(history_vgg19, 'vgg19')

In [ ]:
vgg16_test = vgg16_model.evaluate(val_ds)
vgg19_test = vgg19_model.evaluate(val_ds)

In [ ]:
pd.set_option('precision', 10)
compare_table = pd.DataFrame([vgg16_test,vgg19_test], columns=['loss', 'accuracy'],index=['vgg16', 'vgg19'])
cm = sns.light_palette("green", as_cmap=True)
compare_table.head().style.background_gradient(cmap=cm)

## Prediction

### Select the best model

In [ ]:
if vgg16_test[1] > vgg19_test[1]:
    model = vgg16_model
else:
    model = vgg19_model

### Prepare testing data

In [ ]:
test_filenames = os.listdir(path_to_file + '/dataset/test')
test_df = pd.DataFrame({
    'id': test_filenames
})
nb_samples = test_df.shape[0]
nb_samples

In [ ]:
test_gen = ImageDataGenerator()
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    path_to_file + '/dataset/test', 
    x_col='id',
    y_col=None,
    class_mode=None,
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

### Predict and view

In [ ]:
predict = model.predict(test_generator)
test_df['labels'] = np.argmax(predict, axis=-1)

In [ ]:
test_df

In [ ]:
label_names = {0: 'big', 1: 'normal', 2: 'small'}

fig, ax = plt.subplots(figsize=(15, 7))
i = 0
for index, row in test_df.sample(12).iterrows():
    i += 1
    plt.subplot(3, 4, i)
    img=plt.imread(path_to_file + '/dataset/test/' + row['id'])
    plt.imshow(img)
    plt.axis("off")
    plt.title("label: " + label_names[label.numpy()[i]])
plt.show()

### Output

In [ ]:
test_df.set_index('id')
test_df.to_csv('output.csv', index=False)